In [42]:
import pandas as pd
import numpy as np
import pickle
import os
from tqdm import tqdm

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle

import copy

## Load KNN processed data

In [55]:
RSLT_DIR = './processed/'

In [32]:
col_cand_list=[
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Deaths | Premature Death',
    'Income Inequality | Household Income Inequality',
    'Income | Households with Higher-Incomes',
    'Life Expectancy at Birth | Life Expectancy',
    'Substance Use | Adult Smoking',
    'Births | Preterm Births',
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Cardiovascular Disease | Heart Disease Deaths',
    'Deaths | Infant Deaths',
    'Deaths | Premature Death',
    'Income Inequality | Household Income Inequality',
    'Income | Households with Higher-Incomes',
    'Income | Per-capita Household Income',
    'Language and Nativity | Primarily Speak English',
    'Life Expectancy at Birth | Life Expectancy',
    'Substance Use | Adult Smoking',
    'Cancer | All Cancer Deaths',
    'Cancer | Breast Cancer Deaths',
    'Cancer | Lung Cancer Deaths',
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Cardiovascular Disease | Heart Disease Deaths',
    'Language and Nativity | Primarily Speak Spanish',
    'Births | Teen Births',
    'Cancer | All Cancer Deaths',
    'Cancer | Breast Cancer Deaths',
    'Cancer | Lung Cancer Deaths',
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Cardiovascular Disease | Heart Disease Deaths',
    'Heat and Disasters | Community Social Vulnerability to Climate Disasters',
    'Income-related | Unemployment',
    'Language and Nativity | Primarily Speak Spanish',
    'Mental Health | Suicide',
    'Race/Ethnicity | Minority Population',
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Deaths | Premature Death',
    'Income Inequality | Household Income Inequality',
    'Income | Households with Higher-Incomes',
    'Life Expectancy at Birth | Life Expectancy',
    'Substance Use | Adult Smoking',
    'Births | Preterm Births',
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Cardiovascular Disease | Heart Disease Deaths',
    'Deaths | Infant Deaths',
    'Deaths | Premature Death',
    'Income Inequality | Household Income Inequality',
    'Income | Households with Higher-Incomes',
    'Income | Per-capita Household Income',
    'Language and Nativity | Primarily Speak English',
    'Life Expectancy at Birth | Life Expectancy',
    'Substance Use | Adult Smoking',
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Deaths | Premature Death',
    'Income Inequality | Household Income Inequality',
    'Income | Households with Higher-Incomes',
    'Life Expectancy at Birth | Life Expectancy',
    'Substance Use | Adult Smoking',
    'Births | Preterm Births',
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Cardiovascular Disease | Heart Disease Deaths',
    'Deaths | Infant Deaths',
    'Deaths | Premature Death',
    'Income Inequality | Household Income Inequality',
    'Income | Households with Higher-Incomes',
    'Income | Per-capita Household Income',
    'Language and Nativity | Primarily Speak English',
    'Life Expectancy at Birth | Life Expectancy',
    'Substance Use | Adult Smoking',
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Deaths | Premature Death',
    'Income Inequality | Household Income Inequality',
    'Income | Households with Higher-Incomes',
    'Life Expectancy at Birth | Life Expectancy',
    'Substance Use | Adult Smoking',
    'Births | Preterm Births',
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Cardiovascular Disease | Heart Disease Deaths',
    'Deaths | Infant Deaths',
    'Deaths | Premature Death',
    'Income Inequality | Household Income Inequality',
    'Income | Households with Higher-Incomes',
    'Income | Per-capita Household Income',
    'Language and Nativity | Primarily Speak English',
    'Life Expectancy at Birth | Life Expectancy',
    'Substance Use | Adult Smoking',
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Cardiovascular Disease | Heart Disease Deaths',
    'Deaths | Motor Vehicle Deaths',
    'Deaths | Premature Death',
    'Diabetes and Obesity | Diabetes Deaths',
    'Life Expectancy at Birth | Life Expectancy',
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Cardiovascular Disease | Heart Disease Deaths',
    'Deaths | Gun Deaths (Firearms)',
    'Deaths | Motor Vehicle Deaths',
    'Deaths | Premature Death',
    'Diabetes and Obesity | Diabetes Deaths',
    'Language and Nativity | Foreign Born Population',
    'Life Expectancy at Birth | Life Expectancy',
    'Mental Health | Adult Mental Distress',
    'Racial Segregation Indices | Racial Segregation, White and Non-White',
    'Respiratory Infection | COVID-19 Deaths',
    'Respiratory Infection | New Tuberculosis Cases',
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Cardiovascular Disease | Heart Disease Deaths',
    'Deaths | Motor Vehicle Deaths',
    'Deaths | Premature Death',
    'Diabetes and Obesity | Diabetes Deaths',
    'Life Expectancy at Birth | Life Expectancy',
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Cardiovascular Disease | Heart Disease Deaths',
    'Deaths | Gun Deaths (Firearms)',
    'Deaths | Motor Vehicle Deaths',
    'Deaths | Premature Death',
    'Diabetes and Obesity | Diabetes Deaths',
    'Language and Nativity | Foreign Born Population',
    'Life Expectancy at Birth | Life Expectancy',
    'Mental Health | Adult Mental Distress',
    'Racial Segregation Indices | Racial Segregation, White and Non-White',
    'Respiratory Infection | COVID-19 Deaths',
    'Respiratory Infection | New Tuberculosis Cases',
    'Deaths | Gun Deaths (Firearms)',
    'Deaths | Premature Death',
    'Life Expectancy at Birth | Life Expectancy',
    'Mental Health | Suicide',
    'Substance Use | Drug Overdose Deaths',
    'Substance Use | Opioid Overdose Deaths',
    'Crime Incidents | Homicides',
    'Deaths | Deaths from All Causes',
    'Deaths | Gun Deaths (Firearms)',
    'Deaths | Motor Vehicle Deaths',
    'Deaths | Premature Death',
    'Food Access | Limited Supermarket Access',
    'Language and Nativity | Foreign Born Population',
    'Life Expectancy at Birth | Life Expectancy',
    'Mental Health | Suicide',
    'Sexually Transmitted Disease | New Gonorrhea Cases',
    'Substance Use | Drug Overdose Deaths',
    'Substance Use | Opioid Overdose Deaths',
    'Active Transportation | Walking to Work',
    'Deaths | Deaths from All Causes',
    'Deaths | Premature Death',
    'Education | Preschool Enrollment',
    'Lead Poisoning | Child Lead Levels 5+ mcg/dL',
    'Transportation | Public Transportation Use',
    'Active Transportation | Walking to Work',
    'Deaths | Deaths from All Causes',
    'Deaths | Gun Deaths (Firearms)',
    'Deaths | Infant Deaths',
    'Deaths | Premature Death',
    'Education | Preschool Enrollment',
    'Heat and Disasters | Longer Summers',
    'Income-related | Service Workers',
    'Language and Nativity | Primarily Speak Chinese',
    'Lead Poisoning | Child Lead Levels 5+ mcg/dL',
    'Mental Health | Suicide',
    'Transportation | Public Transportation Use',
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Crime Incidents | Homicides',
    'Deaths | Deaths from All Causes',
    'Deaths | Gun Deaths (Firearms)',
    'Deaths | Injury Deaths',
    'Life Expectancy at Birth | Life Expectancy',
    'Births | Low Birthweight',
    'Births | Preterm Births',
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Cardiovascular Disease | Heart Disease Deaths',
    'Crime Incidents | Homicides',
    'Crime Incidents | Violent Crime',
    'Deaths | Deaths from All Causes',
    'Deaths | Gun Deaths (Firearms)',
    'Deaths | Injury Deaths',
    'Deaths | Maternal Deaths',
    'Life Expectancy at Birth | Life Expectancy',
    'Mental Health | Adult Mental Distress',
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Deaths | Deaths from All Causes',
    'Education | College Graduates',
    'Income Inequality | Income Inequality',
    'Lead Poisoning | Housing Lead Risk',
    'Racial Segregation Indices | Racial Segregation, White and Black',
    'Cardiovascular Disease | Cardiovascular Disease Deaths',
    'Deaths | Deaths from All Causes',
    'Deaths | Racial Disparity in Police Killings',
    'Disability | People with Disabilities',
    'Education | College Graduates',
    'Income Inequality | Income Inequality',
    'Income-related | Service Workers',
    'Lead Poisoning | Housing Lead Risk',
    'Population | Single-Parent Families',
    'Race/Ethnicity | Minority Population',
    'Racial Segregation Indices | Racial Segregation, White and Black',
    'Racial Segregation Indices | Racial Segregation, White and Non-White',
    'Active Transportation | Riding Bike to Work',
    'Education | Preschool Enrollment',
    'Housing | Owner Occupied Housing',
    'Population | Population Density',
    'Race/Ethnicity | Minority Population',
    'Racial Segregation Indices | Racial Segregation, White and Black',
    'Active Transportation | Riding Bike to Work',
    'Education | Preschool Enrollment',
    'Housing | Homeless, Total',
    'Housing | Owner Occupied Housing',
    'Housing | Renters vs. Owners',
    'Income Inequality | Income Inequality',
    'Population | Population Density',
    'Race/Ethnicity | Minority Population',
    'Racial Segregation Indices | Racial Segregation, White and Black',
    'Racial Segregation Indices | Racial Segregation, White and Hispanic',
    'Transportation | Lack of Car',
    'Transportation | Longer Driving Commute Time',
    'Deaths | Injury Deaths',
    'Health Insurance | Uninsured, All Ages',
    'Housing | Homeless, Total',
    'Mental Health | Adult Mental Distress',
    'Sexually Transmitted Disease | Syphilis Prevalence',
    'Sexually Transmitted Disease | Syphilis, Newborns',
    'Deaths | Injury Deaths',
    'Health Insurance | Uninsured, All Ages',
    'Health Insurance | Uninsured, Child',
    'Housing | Homeless, Children',
    'Housing | Homeless, Total',
    'Housing | Vacant Housing and Homelessness',
    'Lead Poisoning | Child Lead Levels 5+ mcg/dL',
    'Mental Health | Adult Mental Distress',
    'Population | Children',
    'Sexually Transmitted Disease | Syphilis Prevalence',
    'Sexually Transmitted Disease | Syphilis, Newborns',
]

info_cols = [
    'city_idx',
    'encoded_strata_race_label',
    'encoded_strata_sex_label',
    'encoded_geo_strata_region',
    'encoded_geo_strata_poverty',
    'encoded_geo_strata_Population',
    'encoded_geo_strata_PopDensity',
    'encoded_geo_strata_Segregation',
    'date_label',
]

In [33]:
def make_reg_score_dict(y_pred,y_actual,base_val):
    rmse_model, rmse_base = np.sqrt(mse(y_actual,y_pred)), np.sqrt(mse([base_val]*len(y_actual),y_actual))
    msle_model, msle_base = 0, 0 #msle(valid_y,y_pred) : negtive value error occurs but i don't know why #msle(valid_y,[train_y.mean()]*len(valid_y)) :
    mape_model, mape_base = mape(y_actual,y_pred), mape([base_val]*len(y_actual),y_actual)
    r2_model, r2_base = r2_score(y_actual,y_pred), 0
    
    return {
        'rmse' : [rmse_model, rmse_base],
        'msle' : [msle_model, msle_base],
        'mape' : [mape_model, mape_base],
        'r2_score' : [r2_model,r2_base]
    }

def print_reg_score_dict(name,dict_score):
    print('{}\nr2 score : {:.5f}'.format(name,dict_score['r2_score'][0]))
    print('rmse_model : {:.5f} / rmse_base : {:.5f}\t'.format(*dict_score['rmse']),
          'mape_model : {:.5f} / mape_base : {:.5f}\t'.format(*dict_score['mape']),
          'msle_model : {:.5f} / msle_base : {:.5f}'.format(*dict_score['msle']))


def make_reg_score_dict_cols(target_sample,dict_df,dict_train_test,dict_rslt,print_plot=False):
    dict_score = dict()
    for col in target_sample:
        train_y = dict_df[col]['train'][1]
        valid_y = dict_train_test[col][3]
        y_pred = dict_rslt[col]['valid']
        dict_score[col] = make_reg_score_dict(y_pred,valid_y,np.mean(train_y))
        if print_plot :
            print_reg_score_dict(col,dict_score[col])
            print('-'*150)
    return dict_score

In [52]:
# entire_label = 
# target_cols = 

knn_score_dict = dict()
base_score_dict = dict()

RSLT_DIR = './processed/'
PVTB_DIR = RSLT_DIR + 'pvtb/'

pvtb_name = 'pvtb_city_encoded_ver1.csv'
pvtb_encoded = pd.read_csv(os.path.join(PVTB_DIR,pvtb_name),index_col=0)

cand_cols = list(set(col_cand_list))
entire_label = list(pvtb_encoded.columns)[9:]
target_cols = list(filter(lambda x : x in cand_cols,entire_label))

geo_name = 'geo_strat_info.csv'
geo_info_path = os.path.join(RSLT_DIR,geo_name)
geo_strat_info = pd.read_csv(geo_info_path, index_col=0)
city_list = list(geo_strat_info.index)

knn_dir = 'knn_rslt'
knn_path = os.path.join(RSLT_DIR,knn_dir)
work_idx = 0
n_work = 10

for work_idx in tqdm(range(0,10)):

    work_name = '{}_{}'.format(work_idx, n_work)
    target_sample = target_cols[work_idx::n_work]
    work_name = '{}_{}'.format(work_idx,n_work) 
    print(f'work : {work_name}')

    pvtb_encoded['city_idx'] = pvtb_encoded['geo_label_city'].apply(lambda x : city_list.index(x))
    temp = list(pvtb_encoded.columns)
    pvtb_encoded_whole = pvtb_encoded[['city_idx']+temp[1:-1]]
    test_df = pvtb_encoded[info_cols+target_cols]


    files = list(filter(lambda x : work_name in x,os.listdir(knn_path)))
    file_dict = dict()
    for filename in files:
        if filename[-4:] != '.pkl' : continue
        if 'knn' in filename : continue
        print(filename)
        file_path = os.path.join(knn_path,filename)
        with open (file_path,'rb') as f:
            file_dict[filename[:-5-len(work_name)]] = pickle.load(f)
    dict_rslt = file_dict['dict_rslt']
    target_sample= list(dict_rslt.keys())
    # display(test_df.columns)
    rslt_form = test_df[info_cols+target_sample]

    for col in target_sample:
        cond = test_df[col].isna()
        loaded = dict_rslt[col]['target']
        assert np.sum(cond) == len(loaded)  
        rslt_form.loc[cond,col] = loaded

    # file_name = 'pvtb_filled_knn_{}_{}.csv'.format(work_idx,10)
    # save_dir = os.path.join(RSLT_DIR,knn_dir,'PROCESSED')
    # if not os.path.exists(save_dir): os.mkdir(save_dir)
    # rslt_form.to_csv(os.path.join(save_dir,file_name))

    ### check score for loaded data

    dict_rslt=file_dict['dict_rslt']
    dict_train_test=file_dict['dict_train_test']

    dict_df = dict()
    for col in target_sample:
        temp = test_df[info_cols+[col]]
        cond_na = temp.isna().any(axis=1)
        dict_df[col] = {
            'train' : [temp.loc[~cond_na,info_cols], temp.loc[~cond_na,col]],
            'target' : [temp.loc[cond_na,info_cols], cond_na],
        }

    ## check score
    dict_score = make_reg_score_dict_cols(copy.deepcopy(target_sample),dict_df.copy(),copy.deepcopy(dict_train_test),dict_rslt,print_plot=False)

    knn_score = copy.deepcopy(dict_score)
    base_score = copy.deepcopy(dict_score)
    for key in dict_score.keys():
        for key2 in dict_score[key].keys():
            knn_score[key][key2] = knn_score[key][key2][0]
            base_score[key][key2] = base_score[key][key2][1]

    knn_score_dict = knn_score_dict | knn_score
    base_score_dict = base_score_dict | base_score
    # print(dict_score)
    # knn_df = pd.DataFrame(knn_score)
    # base_df = pd.DataFrame(base_score)
    # # display(base_score_df)

    # knn_score_df = pd.merge(knn_score_df)
    # knn_scores_df
    
    # save_dir = os.path.join(RSLT_DIR,knn_dir,'PLOT')
    # if not os.path.exists(save_dir): os.mkdir(save_dir)

    # fig,axes = scatter_reg_rslt(dict_train_test,dict_rslt)
    # file_name = 'reg_scatter_{}.png'.format(work_name)
    # fig.savefig(os.path.join(save_dir,file_name))

    # fig,axes = plot_reg_score(dict_train_test,dict_rslt,dict_score)
    # file_name = 'reg_rslt_{}.png'.format(work_name)
    # fig.savefig(os.path.join(save_dir,file_name))

print("process done")

  0%|          | 0/10 [00:00<?, ?it/s]

 20%|██        | 2/10 [00:00<00:00, 11.46it/s]

work : 0_10
dict_rslt_0_10.pkl
dict_train_test_0_10.pkl
work : 1_10
dict_rslt_1_10.pkl
dict_train_test_1_10.pkl
work : 2_10
dict_rslt_2_10.pkl
dict_train_test_2_10.pkl


 40%|████      | 4/10 [00:00<00:00, 10.84it/s]

work : 3_10
dict_rslt_3_10.pkl
dict_train_test_3_10.pkl
work : 4_10
dict_rslt_4_10.pkl
dict_train_test_4_10.pkl
work : 5_10
dict_rslt_5_10.pkl
dict_train_test_5_10.pkl


 80%|████████  | 8/10 [00:00<00:00, 10.39it/s]

work : 6_10
dict_rslt_6_10.pkl
dict_train_test_6_10.pkl
work : 7_10
dict_rslt_7_10.pkl
dict_train_test_7_10.pkl
work : 8_10
dict_rslt_8_10.pkl
dict_train_test_8_10.pkl
work : 9_10
dict_rslt_9_10.pkl
dict_train_test_9_10.pkl


100%|██████████| 10/10 [00:01<00:00,  8.90it/s]

process done


In [54]:
knn_df = pd.DataFrame(knn_score_dict)
base_df = pd.DataFrame(base_score_dict)


In [69]:
knn_score_dict

for key in knn_score_dict.keys():
    temp = knn_score_dict[key]
    knn_score_dict[key] = dict()
    knn_score_dict[key][True] = temp
    knn_score_dict[key][False] = temp

for key in base_score_dict.keys():
    temp = base_score_dict[key]
    base_score_dict[key] = dict()
    base_score_dict[key][True] = temp
    base_score_dict[key][False] = temp

## LOAD RFC DATA

In [63]:
rfc_knn = dict()
for n_est in [50,100,500]:
    for h in [True, False]:
        suffix = "_h" if h else ""
        with open(f"./results/rf/result_rf_{n_est}{suffix}.pickle",'rb') as file:
            score = pickle.load(file)
            if n_est not in rfc_knn.keys():
                rfc_knn[n_est] = dict()
            rfc_knn[n_est][h]= score

rfc_orig = dict()
for n_est in [50,100,500]:
    for h in [True, False]:
        suffix = "_h" if h else ""
        with open(f"./results/rf/result_reindex_{n_est}{suffix}.pickle",'rb') as file:
            score = pickle.load(file)
            if n_est not in rfc_orig.keys():
                rfc_orig[n_est] = dict()
            rfc_orig[n_est][h]= score

## Load XGB Data

In [65]:
# for lr in [0.01,0.05,0.1]:
#     for n_est in [100,300,500]:
#         for h in [True, False]:
#             suffix = "_h" if h else ""
#             with open(f"./results/xgb/result_xgb_{lr}_{n_est}{suffix}.pickle",'rb') as file:
#                 score = pickle.load(file)
#                 print(score)

## LOAD MLP DATA

In [61]:
with open("results/mlp/mlp.pickle",'rb') as file:
    mlp_knn = pickle.load(file)

with open("results/mlp/mlp_reindex.pickle",'rb') as file:
    mlp_orig = pickle.load(file)

## COMBINE

In [70]:
combined_scores = dict()
combined_scores["MLP-KNN"] = mlp_knn
combined_scores["MLP"] = mlp_orig
combined_scores["RFC-KNN"] = rfc_knn
combined_scores["RFC"] = rfc_orig
combined_scores["KNN"] = knn_score_dict
combined_scores["Base"] = base_score_dict
# combined_scores["XGB-KNN"]
# combined_scores["RFC-KNN"]


In [88]:
def flatten_dict(nested_dict):
    res = {}
    if isinstance(nested_dict, dict):
        for k in nested_dict:
            flattened_dict = flatten_dict(nested_dict[k])
            for key, val in flattened_dict.items():
                key = list(key)
                key.insert(0, k)
                res[tuple(key)] = val
    else:
        res[()] = nested_dict
    return res


def nested_dict_to_df(values_dict):
    flat_dict = flatten_dict(values_dict)
    df = pd.DataFrame.from_dict(flat_dict, orient="index")
    df.index = pd.MultiIndex.from_tuples(df.index)
    print(df.index)
    df = df.unstack(level=-1)
    df.columns = df.columns.map("{0[1]}".format)
    return df

In [94]:
for key in score_dict[True].keys():
    print(score_dict[True][key].keys())

dict_keys([0.01, 0.001, 0.0001])
dict_keys([0.01, 0.001, 0.0001])
dict_keys([50, 100, 500])
dict_keys([50, 100, 500])
dict_keys(['Diabetes and Obesity | Diabetes Deaths', 'Deaths | Premature Death', 'Lead Poisoning | Housing Lead Risk', 'Language and Nativity | Foreign Born Population', 'Respiratory Infection | New Tuberculosis Cases', 'Substance Use | Drug Overdose Deaths', 'Health Insurance | Uninsured, Child', 'Deaths | Racial Disparity in Police Killings', 'Language and Nativity | Primarily Speak Chinese', 'Crime Incidents | Homicides', 'Food Access | Limited Supermarket Access', 'Active Transportation | Walking to Work', 'Mental Health | Adult Mental Distress', 'Substance Use | Opioid Overdose Deaths', 'Deaths | Maternal Deaths', 'Housing | Homeless, Children', 'Births | Low Birthweight', 'Health Insurance | Uninsured, All Ages', 'Sexually Transmitted Disease | New Gonorrhea Cases', 'Deaths | Deaths from All Causes', 'Crime Incidents | Violent Crime', 'Cancer | All Cancer Deaths',

In [ ]:
 heuristic
mode, feature, hyperparam,, value

In [90]:
score_dict = dict()
for key in combined_scores.keys():
    for key2 in combined_scores[key].keys():
        for key3 in combined_scores[key][key2].keys():
            score_dict[key3] = copy.deepcopy(combined_scores)

for key in score_dict[True].keys():
    score_dict[True][key] = score_dict[True][key][True]

for key in score_dict[False].keys():
    score_dict[False][key] = score_dict[True][key][False]


KeyError: True

In [89]:
import seaborn as sns
import matplotlib.pyplot as plt


# fig ,axes = plt.subplots(6,1,figsize=(16,32))

combined_scores.keys()
df_dict = nested_dict_to_df(combined_scores)

# df_dict


# for idx, key in enumerate(combined_scores.keys()):
#     data_dict = combined_scores[key]
#     keys = data_dict.keys()

#     df_dict = nested_dict_to_df(combined_scores).reset_index()
#     display(df_dict)
#     # df_dict = df_dict.rename(columns={"level_0": "Heuristics","level_2": "mode"})
#     df_dict = df_dict[df_dict["mode"] == "train"]
#     # pd.DataFrame.from_dict()

#     sns.barplot(data=df_dict,x='level_1', y='rmse', hue = 'Heuristics', ax=axes[idx])
#     axes[idx].set_xticks(axes[idx].get_xticks(), axes[idx].get_xticklabels(), rotation=90, ha='right')

#     axes[idx].set(xlabel="Features",ylabel="RMSE-score",title=f"RMSE for lr={key}")

#     for p in axes[idx].patches:
#         axes[idx].annotate(f"{p.get_height():.2f}", (p.get_x() + p.get_width() / 2., p.get_height()),
#                     ha='center', va='center', xytext =(0, 7), textcoords='offset points')

IndexError: tuple index out of range